## Car Evaluation Model

Descrição: O dataset contém diversas informações sobre carros, as classes unacc (unaacceptable), acc (acceptable), good, vgood (very good) e um total de 1728 intâncias

### Imports

In [34]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


### Carregando o Dataset Car Evaluation

In [35]:
car_evaluation = fetch_ucirepo(id=19)
X_original = car_evaluation.data.features
y_original = car_evaluation.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")


Dimensão inicial de X: (1728, 6)
Colunas de X e seus tipos de dados:
buying      object
maint       object
doors       object
persons     object
lug_boot    object
safety      object
dtype: object

Primeiras 5 linhas de X:
  buying  maint doors persons lug_boot safety
0  vhigh  vhigh     2       2    small    low
1  vhigh  vhigh     2       2    small    med
2  vhigh  vhigh     2       2    small   high
3  vhigh  vhigh     2       2      med    low
4  vhigh  vhigh     2       2      med    med

Classes de y:
class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64


### Pré-processamento

In [36]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False retorna array denso
# sparse_output=False é o mesmo que chamar .toarray() depois.
X_encoded_dense = encoder.fit_transform(X_original)

print(f"Dimensão de X após One-Hot Encoding: {X_encoded_dense.shape}")
print(f"Tipo dos elementos após One-Hot Encoding: {X_encoded_dense.dtype}")
print(f"Primeiras 2 linhas de X_encoded_dense:\n{X_encoded_dense[:2]}")
print("-" * 50)

Dimensão de X após One-Hot Encoding: (1728, 21)
Tipo dos elementos após One-Hot Encoding: float64
Primeiras 2 linhas de X_encoded_dense:
[[0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]]
--------------------------------------------------


### Dividindo os Dados em Treino e Teste

In [37]:
# Para o WiSARD, os dados de entrada X devem ser do tipo np.uint8.
# E os rótulos y devem ser strings.
X_encoded_uint8 = X_encoded_dense.astype(np.uint8)
y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")


Tamanho do conjunto de treino (X_train): 518 amostras, 21 bits
Tamanho do conjunto de teste (X_test): 1210 amostras, 21 bits
Tipo dos elementos de X_train: uint8


### Treinando o Modelo WiSARD

In [38]:
addressSize = 8 

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)

predictions_str = wsd.classify(X_test)

y_pred_np = np.array(predictions_str)
y_test_np = np.array(y_test) 

WiSARD inicializado com addressSize=8
training 518 of 51818
classifying 453 of 1210

classifying 1210 of 1210


### Relatório de Classificação

In [39]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

         acc       0.69      0.48      0.56       269
        good       0.00      0.00      0.00        48
       unacc       0.83      0.97      0.90       847
       vgood       0.75      0.46      0.57        46

    accuracy                           0.80      1210
   macro avg       0.57      0.48      0.51      1210
weighted avg       0.76      0.80      0.77      1210



### WiSARD X MLP

Embora tenha executado muito mais rápido que a versão em MLP houve uma perda significativa nas métricas de avaliação